In [ ]:
pip install -U langgraph langsmith langchain langchain-google-genai google-generativeai

In [ ]:
import os
os.environ["GOOGLE_API_KEY"] = "GOOGLE_API_KEY"  # Replace with your Gemini API key

In [ ]:
from typing import Annotated
from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain_google_genai import ChatGoogleGenerativeAI

# Define the state
class State(TypedDict):
    messages: Annotated[list, add_messages]

# Initialize Gemini 2.0 Flash LLM


llm = ChatGoogleGenerativeAI(
    model="models/gemini-2.0-flash",
    google_api_key=os.environ["GOOGLE_API_KEY"]
)

# Build the LangGraph
graph_builder = StateGraph(State)

# You would add LLM logic here as a node later
# Example:
# graph_builder.add_node("gemini_node", some_function_that_calls_llm)

In [ ]:
from langgraph.graph import START, END
from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages
from typing import Annotated
from typing_extensions import TypedDict
import os

# ✅ Define the state structure
class State(TypedDict):
    messages: Annotated[list, add_messages]

# ✅ Initialize the LLM with Gemini 2.0 Flash
llm = ChatGoogleGenerativeAI(
    model="models/gemini-2.0-flash",
    google_api_key=os.environ["GOOGLE_API_KEY"]
)

# ✅ Define the chatbot function for the graph node
def chatbot(state: State) -> dict:
    return {"messages": [llm.invoke(state["messages"])]}

# ✅ Build the graph
graph_builder = StateGraph(State)
graph_builder.add_node("chatbot", chatbot)

In [ ]:
# ✅ Install dependencies (run only once)
!pip install -U langchain langgraph langchain-google-genai

# ✅ Import necessary packages
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from typing_extensions import TypedDict
from typing import Annotated
from langchain_core.messages import HumanMessage
import getpass

# ✅ Securely enter your Gemini API key
os.environ["GOOGLE_API_KEY"] = getpass.getpass("🔐 Enter your Gemini API key: ")

# ✅ Define conversation state
class State(TypedDict):
    messages: Annotated[list, add_messages]

# ✅ Initialize Gemini 2.0 Flash model
llm = ChatGoogleGenerativeAI(
    model="models/gemini-2.0-flash",
    google_api_key=os.environ["GOOGLE_API_KEY"]
)

# ✅ Define chatbot behavior
def chatbot(state: State):
    return {"messages": [llm.invoke(state["messages"])]}

# ✅ Build LangGraph
graph_builder = StateGraph(State)
graph_builder.add_node("chatbot", chatbot)
graph_builder.add_edge(START, "chatbot")
graph_builder.add_edge("chatbot", END)
graph = graph_builder.compile()

# ✅ Function to stream Gemini chat
def stream_graph_updates(user_input: str):
    for event in graph.stream({"messages": [HumanMessage(content=user_input)]}):
        for value in event.values():
            print("🤖 Assistant:", value["messages"][-1].content)

# ✅ Start interactive chat
while True:
    try:
        user_input = input("🧑 You: ")
        if user_input.lower() in ["quit", "exit", "q"]:
            print("👋 Goodbye!")
            break
        stream_graph_updates(user_input)
    except Exception as e:
        print("⚠ Error:", e)
        break